In [58]:
import os

from azure.ai.ml import MLClient
from azure.ai.ml.entities import Environment, AmlCompute 
from azure.identity import InteractiveBrowserCredential

from load_secrets import load_secrets

In [23]:
tenant_id = load_secrets('secrets.ini')["tenant_id"]

In [27]:
credential = InteractiveBrowserCredential(tenant_id=tenant_id)

In [28]:
client = MLClient.from_config(credential=credential,
    file_name="./azure-config.json")

Found the config file in: .\./azure-config.json


In [30]:
workspace = client.workspaces.get()
print(workspace.name)

mlops-e2e-01


In [121]:
client.workspaces.get(client.workspace_name).mlflow_tracking_uri

'azureml://centralindia.api.azureml.ms/mlflow/v1.0/subscriptions/6202d817-1165-4812-9031-c0f106108096/resourceGroups/mlops-e2e/providers/Microsoft.MachineLearningServices/workspaces/mlops-e2e-01'

In [33]:
dependencies_dir = "./dependencies"
os.makedirs(dependencies_dir, exist_ok=True)

In [41]:
# %%writefile {dependencies_dir}/conda.yaml
# name: model-env
# channels:
#   - conda-forge
# dependencies:
#   - python=3.11.5
#   - pip
#   - pip:
#     - -r ./requirements.txt

Overwriting ./dependencies/conda.yaml


In [125]:
env_name = "mlopse2eenv"

env = Environment(
    name=env_name,
    conda_file=os.path.join("dependencies", "conda.yaml"),
    # image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest"
    image="mcr.microsoft.com/azureml/curated/mldesigner:24"
)

env = client.environments.create_or_update(env)

In [126]:
print(f"Environment with name {env.name} is registered to workspace, the environment version is {env.version}")

Environment with name mlopse2eenv is registered to workspace, the environment version is 2


In [127]:
next(client.compute.list()).name

'compute01'

In [128]:
from azure.ai.ml import command
from azure.ai.ml import Input


In [129]:
train_inputs = {
    "data": Input(
        type="uri_file",
        path="azureml://subscriptions/6202d817-1165-4812-9031-c0f106108096/resourcegroups/mlops-e2e/workspaces/mlops-e2e-01/datastores/workspaceblobstore/paths/UI/2024-05-08_093653_UTC/raw_data/",
    ),
    "epochs": 2
}

In [130]:
train_job = command(
    # name="train job",
    # description="Train job for MLOPS E2E",
    # display_name="train job",
    inputs=train_inputs,
    code="./src",
    command="python ./pipeline/train_pipeline.py --data ${{inputs.data}}",
    compute="compute01",
    experiment_name="mlopse2e",
    environment=f"{env.name}@latest",
)

In [131]:
client.create_or_update(train_job)

Experiment,Name,Type,Status,Details Page
mlopse2e,loving_leg_twr8bkwt8n,command,Starting,Link to Azure Machine Learning studio
